In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
import numpy as np
import pandas as pd
import gensim
import pickle
from scipy.stats import pearsonr, spearmanr
import tensorflow as tf
import matplotlib.pyplot as plt
import re
import itertools

In [ ]:
df_train = pd.read_csv('./data/semeval_train_ko.csv')
df_train_norm = pd.read_csv('./data/semeval_train.csv')
df_test = pd.read_csv('./data/semeval_test_ko.csv')

# df_test = df_test.iloc[:len(df_test)-1, :]
df_test.tail()

In [ ]:
df_train.tail()

In [ ]:
df_train_norm['sent_1'] = df_train_norm['sentence_A']
df_train_norm['sent_2'] = df_train_norm['sentence_B']
df_train_norm['sim'] = df_train_norm['relatedness_score']
df_train_norm = df_train_norm[['sent_1', 'sent_2', 'sim']]

In [ ]:
df_test_norm = pd.read_csv('./data/semeval_test.csv')
df_test_norm['sim'] = norm(df_test_norm['relatedness_score'])
df_test_norm['sent_1'] =df_test_norm['sentence_A']
df_test_norm['sent_2'] = df_test_norm['sentence_B']
df_test_norm = df_test_norm[['sent_1', 'sent_2', 'sim']]
df_test_norm.tail()

In [ ]:
def norm(x):
    return (x - np.min(x)) / (np.max(x) - np.min(x)) 

In [ ]:
df_test['sent_1'] = df_test['sentence_A']
df_test['sent_2'] = df_test['sentence_B']

# df_train['sent_1'] = df_train['sentence_A']
# df_train['sent_2'] = df_train['sentence_B']

In [ ]:
df_train = df_train.append(df_train_norm)
df_test = df_test.append(df_test_norm)

df_train = df_train.reset_index()
df_test = df_test.reset_index()

In [ ]:
def clean_text(text):
    ''' Pre process and convert texts to a list of words '''
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    return text


def text_to_word_list(text):
    ''' Pre process and convert texts to a list of words '''
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    text = text.split()

    return text


In [ ]:
def prepare_embeddings(model, datasets, question_cols):
    vocabulary = dict()
    inverse_vocabulary = ['<unk>']
    questions_cols = question_cols

    # Iterate over the questions only of both training and test datasets
    for dataset in datasets:
        for index, row in dataset.iterrows():

            # Iterate through the text of both questions of the row
            for question in questions_cols:

                q2n = []  # q2n -> question numbers representation
                for word in text_to_word_list(row[question]):

                    # # Check for unwanted words
                    if word not in model.vocab:
                        continue

                    if word not in vocabulary:
                        vocabulary[word] = len(inverse_vocabulary)
                        q2n.append(len(inverse_vocabulary))
                        inverse_vocabulary.append(word)
                    else:
                        q2n.append(vocabulary[word])

                # Replace questions as word to question as number representationindex, question, q2n
                dataset.at[index, question]= q2n

    embedding_dim = model.vector_size
    embeddings = 1 * np.random.randn(len(vocabulary) + 1, embedding_dim)  # This will be the embedding matrix
    embeddings[0] = 0  # So that the padding will be ignored

    # Build the embedding matrix
    for word, index in vocabulary.items():
        if word in model.vocab:
            embeddings[index] = model.word_vec(word)

    return embeddings, embedding_dim

In [ ]:
# word2vec = gensim.models.KeyedVectors.load_word2vec_format('./models/enwiki_20180420_300d.txt')
word2vec = gensim.models.KeyedVectors.load_word2vec_format('./models/GoogleNews-vectors-negative300.bin', binary=True)

In [ ]:
embeddings, embedding_dim = prepare_embeddings(model=word2vec, datasets=[df_train, df_test], question_cols=['sent_1', 'sent_2'])

In [ ]:
max_seq_length = max(df_train.sent_1.map(lambda x: len(x)).max(),
                     df_train.sent_2.map(lambda x: len(x)).max(),
                     df_test.sent_1.map(lambda x: len(x)).max(),
                     df_test.sent_2.map(lambda x: len(x)).max())

X_train = {'left': df_train.sent_1, 'right': df_train.sent_2}
X_test = {'left': df_test.sent_1, 'right': df_test.sent_2}

for dataset, side in itertools.product([X_train, X_test], ['left', 'right']):
        dataset[side] = tf.keras.preprocessing.sequence.pad_sequences(dataset[side], maxlen=max_seq_length)

In [ ]:
def exponent_neg_manhattan_distance(left, right):
    """ Helper function for the similarity estimate of the RNNs outputs"""
    return tf.keras.backend.exp(-tf.keras.backend.sum(tf.keras.backend.abs(left - right), 
                                                      axis=1, keepdims=True))


left_input = tf.keras.layers.Input(shape=(max_seq_length,), dtype='int32')
right_input = tf.keras.layers.Input(shape=(max_seq_length,), dtype='int32')

embedding_layer = tf.keras.layers.Embedding(len(embeddings), 
                                            embedding_dim, 
                                            weights=[embeddings], 
                                            input_length=max_seq_length,
                                            trainable=False)

encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

# Since this is a siamese network, both sides share the same RNN
# shared_gru = tf.keras.layers.GRU(100, name='gru', recurrent_activation='sigmoid', reset_after=True,
#                                 bias_initializer=tf.keras.initializers.Constant(2.5), dropout=0.0,
#                                 kernel_regularizer=None, recurrent_dropout=0.0)

# Since this is a siamese network, both sides share the same RNN
shared_gru = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(100, name='gru', recurrent_activation='sigmoid', reset_after=True,
                                                               bias_initializer=tf.keras.initializers.Constant(2.5), dropout=0.0,
                                                               kernel_regularizer=None, recurrent_dropout=0.0))


shared_dense = tf.keras.layers.Dense(50, activation='relu')
dp = tf.keras.layers.Dropout(0.25)

left_output = shared_gru(encoded_left)
right_output = shared_gru(encoded_right)

# left_output_den = shared_dense(left_output)
# right_output_den = shared_dense(right_output)

# left_output_dp = dp(left_output_den)
# right_output_dp = dp(right_output_den)

# Calculates the distance as defined by the MaLSTM model
magru_distance = tf.keras.layers.Lambda(
    function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]),
    output_shape=lambda x: (x[0][0], 1))([left_output, right_output])

magru = tf.keras.Model([left_input, right_input], [magru_distance])
optimizer=tf.keras.optimizers.Adadelta(learning_rate=1, rho=0.985, clipvalue=2.0)
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.01, clipvalue=1.5)

magru.compile(loss='mean_squared_error', optimizer=optimizer)

In [ ]:
hist = magru.fit([X_train['left'], X_train['right']], 
                np.array(df_train['sim']), 
                epochs=500, 
                batch_size=64,
                validation_data=([X_test['left'], X_test['right']], df_test['sim'])
                )

In [ ]:
preds = magru.predict([X_train['left'], X_train['right']])
print(f"Train: {pearsonr([x[0] for x in preds.tolist()], df_train['sim'])[0]}")
preds = magru.predict([X_test['left'], X_test['right']])
print(f"Test: {pearsonr([x[0] for x in preds.tolist()], df_test['sim'])[0]}")

In [ ]:
# If we are using TTA, please use the following...
b_idx = 0
e_idx = 4927

p_avg = list()
while e_idx < len(preds):
    first = preds[b_idx]
    second = preds[e_idx]
    p_avg.append(np.mean([first, second]))
    b_idx += 1
    e_idx += 1
    
pearsonr(p_avg, df_test_norm['sim'])